Data description at the [codalab 26655 competition](https://competitions.codalab.org/competitions/26655#learn_the_details).


In [ ]:
!pip install transformers bertopic umap

# Imports

In [ ]:
import pandas as pd
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
from bertopic.representation import MaximalMarginalRelevance
from bertopic.vectorizers import ClassTfidfTransformer
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Data loading

In [ ]:
train = pd.read_excel("data/Constraint_English_Train.xlsx")
val = pd.read_excel("data/Constraint_English_Val.xlsx")
test = pd.read_excel("data/Constraint_English_Test.xlsx")
data = pd.concat([train, val, test], ignore_index=True)

In [ ]:
train.head()

In [ ]:
val.head()

In [ ]:
test.head()

In [ ]:
data.head()

# EDA

Initially, we check the data type of the columns of the dataframe `data`.

In [ ]:
data.info()

As we can see, each row contains an *id*, a *tweet* (which corresponds to a string) and a *label*. Next, let's show the distribution of the tags in the dataframe to check for class imbalance.

In [ ]:
data["label"].value_counts().plot(kind="bar", color=["blue", "red"])
plt.title("Label Distribution")
plt.xlabel("Label")
plt.ylabel("Count")
plt.show()

According to the graph above, the number of tweets labeled as “real” are slightly higher than those labeled as “fake”, although the percentage is quite similar and does not pose any class imbalance problem.

Another important thing when analyzing this dataset is to check which are the most repeated words in the *real* tweets and in the *fake* tweets. To do this, we are going to use `WordCloud()` to visualize the frequency of words in each category. This will give us insights into the key terms used in tweets labeled as *real* and *fake* and help us understand potential linguistic patterns or biases in the dataset.

In [ ]:
# Plotting the word cloud for real tweets
plt.imshow(WordCloud(width=800, height=800, background_color="white").generate(" ".join(data[data["label"] == "real"]["tweet"])))
plt.axis("off")
plt.title("Word Cloud (Real tweets)")
plt.show()

In [ ]:
# Plotting the word cloud for fake tweets
plt.imshow(WordCloud(width=800, height=800, background_color="white").generate(" ".join(data[data["label"] == "fake"]["tweet"])))
plt.axis("off")
plt.title("Word Cloud (Fake tweets)")
plt.show()

# Preprocessing

In this section, data will be preprocessed and other important features will be extracted from the initial information in order to more fully and rigorously analyze the data.

### Tweet length

Next, we get some other interesting data such as the average length of tweets according to their label.

In [ ]:
data['tweet_length'] = data['tweet'].apply(len)
data.head()

In [ ]:
# Show a tweet text and the corresponding label
n = 527

print(data["tweet"][n])
print(data["label"][n])

Outliers will be removed only to check the distribution according to the length of the tweet as a function of its label. In this way, we will be able to study if there is any relationship between the length of the tweet and its label without taking into account the noise that may be caused by the use of outliers.

In [ ]:
# Remove the outliers
Q1 = data['tweet_length'].quantile(0.25)
Q3 = data['tweet_length'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

data_tweet_length_outliers_removed = data[(data['tweet_length'] > lower_bound) & (data['tweet_length'] < upper_bound)]

In [ ]:
# Plot the distribution of training real and fake tweets
plt.figure(figsize=(12, 12))

# Plot with outliers removed
plt.subplot(2, 2, 1)
plt.hist(data_tweet_length_outliers_removed[data_tweet_length_outliers_removed['label'] == 'real']['tweet_length'], bins=35, label='Real', color='blue', alpha=0.6)
plt.title('Real Tweets with outliers removed')
plt.xlabel('Tweet Length')
plt.ylabel('Frequency')
plt.legend()

plt.subplot(2, 2, 2)
plt.hist(data_tweet_length_outliers_removed[data_tweet_length_outliers_removed['label'] == 'fake']['tweet_length'], bins=35, label='Fake', color='red', alpha=0.6)
plt.title('Fake Tweets with outliers removed')
plt.xlabel('Tweet Length')
plt.ylabel('Frequency')
plt.legend()

# Plot with outliers
plt.subplot(2, 2, 3)
plt.hist(data[data['label'] == 'real']['tweet_length'], bins=35, label='Real', color='blue', alpha=0.6)
plt.title('Real Tweets with outliers')
plt.xlabel('Tweet Length')
plt.ylabel('Frequency')
plt.legend()

plt.subplot(2, 2, 4)
plt.hist(data[data['label'] == 'fake']['tweet_length'], bins=35, label='Fake', color='red', alpha=0.6)
plt.title('Fake Tweets with outliers')
plt.xlabel('Tweet Length')
plt.ylabel('Frequency')
plt.legend()

plt.tight_layout()
plt.show()

To validate this hypothesis with data, we will calculate the skewness of the length of tweets as a function of their label. In addition, we will also analyze other interesting metrics such as the mean and median tweet length.

In [ ]:
# Data skewness
data_real_skew = data_tweet_length_outliers_removed[data_tweet_length_outliers_removed['label'] == 'real'].tweet_length.skew()
data_fake_skew = data_tweet_length_outliers_removed[data_tweet_length_outliers_removed['label'] == 'fake'].tweet_length.skew()
print("\n-----DATA-----")
print(f"Real skewness: {data_real_skew}")
print(f"Fake skewness: {data_fake_skew}")

In [ ]:
# Data real tweet_length mean and median
data_real_mean = data_tweet_length_outliers_removed[data_tweet_length_outliers_removed['label'] == 'real'].tweet_length.mean()
data_real_median = data_tweet_length_outliers_removed[data_tweet_length_outliers_removed['label'] == 'real'].tweet_length.median()
print(f"Label: 'real', Mean: {data_real_mean}, Median: {data_real_median}")

# Data fake tweet_length mean and median
data_fake_mean = data_tweet_length_outliers_removed[data_tweet_length_outliers_removed['label'] == 'fake'].tweet_length.mean()
data_fake_median = data_tweet_length_outliers_removed[data_tweet_length_outliers_removed['label'] == 'fake'].tweet_length.median()
print(f"Label: 'fake', Mean: {data_fake_mean}, Median: {data_fake_median}")

We can draw the following conclusions about the `tweet_lenght`:

- Tweets labeled `real` have a moderate negative skewness, which means that the distribution is shifted to the left and therefore longer tweets are more frequent. To be more precise, the mean number of characters used in tweets labeled `real` is 215.00 and the median 227.0.

- The tweets labeled `fake` have a high positive skewness, which means that the distribution is quite shifted to the right and, therefore, shorter tweets are more frequent. To be more precise, the mean number of characters used in `fake` labeled tweets is 137.90 and the median 118.0.

# BERTopic

## Embeddings

In [ ]:
embedding_model = SentenceTransformer("abhinand/MedEmbed-large-v0.1") # El inicial era ""sentence-transformers/all-MiniLM-L6-v2"
embeddings = embedding_model.encode(train["tweet"])

There are different models to obtain embeddings, some of them are:

- **Sentence Transformers**: `Sentence Transformers` are neural network-based models with `Transformer` architecture designed to create dense vector representations (embeddings) of text. These models convert sentences, paragraphs or documents into vectors of fixed size $v \in \mathbb{R}^d$, where $d$ is the dimension of the vector space. The vector representation $v$ captures the semantic meaning, allowing operations such as similarity matching and clustering.


- **TF-IDF**: Term Frequency-Inverse Document Frequency (TF-IDF) embeddings are a statistical technique used to represent words in text based on their frequency within a document and their rarity in a corpus. It is calculated as the product of two components:

    1. **Term Frequency (TF)**: The frequency of a term $t$ in a document $d$, calculated as:
        $$\text{TF}(t, d) = \frac{\text{number of times t appears in d}}{\text{total number of terms in d}}$$

    2. **Inverse Document Frequency (IDF)**: The occurrence of a term $t$ e in all documents in the collection. Rarer terms that appear in fewer documents get higher **IDF score**. It can be calculated as follows:
    $$\text{IDF}(t) = \log\left(\frac{N}{1 + \text{df}}\right)$$
        
    The **embedding TF-IDF** of a term in a document is obtained by multiplying both values:
    $$\text{TF-IDF}(t,d) = \text{TF}(t,d) \times \text{IDF}(t)$$


- **Word2Vec**: `Word2Vec` is a neural network-based model that learns to represent words as dense vectors in a high-dimensional space. This model is trained using two main architectures: **Skip-gram** and **Continuous Bag of Words (CBOW)**. Both are based on the prediction of words based on their context.

    1. **Skip-gram**: Given a context $C = \{w1,w2,...,w_{m}\}$, where $w_{1}$ is the target word, the model tries to predict the context words from the target word $w_{1}$. The probability that a word $w$ is in the given context $w_{1}$ is modeled as:
    $$P(w | w_1) = \frac{\exp(\mathbf{v'}_{w_0}^T \mathbf{v}_{w_1})}{\sum_{w=1}^{W} \exp(\mathbf{v'}_{w}^T \mathbf{v}_{w_1})}$$
     
    where $v'_{w_0}$ y $v_{w_1}$ are the word vectors for $w_0$ and $w_1$ respectively, and $W$ is the total vocabulary.

    2. **CBOW (COntinous Bag of Words)**: (Still to be completed)

    In both cases, the model seeks to maximize the probability that words in the context of a target word are correctly represented, resulting in vector representations of words (embeddings).

    The resulting **Word2Vec** embeddings capture semantic relationships between words, so that words with similar meanings are grouped together in the vector space. For example, the relationship “king - man + man + woman = queen” can be approximated by operations in vector space, reflecting semantic relationships. In addition, Word2Vec does not take into account the global context of words in a complete text. The model learns semantic relationships based on local patterns in the text and does not take into account global information such as the overall topic of a document or the meaning a word may acquire depending on its position within the full text.

## Dimensionality Reduction

In [ ]:
umap_model = UMAP(n_neighbors=20, n_components=10, metric='cosine', random_state=42, low_memory=False) # PCA

For this problem, we will use UMAP, since the embeddings obtained using the SentenceTransformer model present a nonlinear structure. UMAP is particularly suitable for this type of data, as it allows preserving complex local relationships by focusing on the neighborhood topology in the high dimensionality space. Unlike PCA, which identifies directions of maximum variance and is more effective for data with linear structures, UMAP captures more subtle and nonlinear local patterns, making it ideal for analyzing complex semantic relationships in the generated embeddings.

> **Source**: https://www.proquest.com/docview/2473295334?pq-origsite=gscholar&fromopenview=true&sourcetype=Scholarly%20Journals
"PCA projection identifies directions of maximal variance in the data and ignores variation along other directions, it tends to obscure finer-scale patterns of population structure. Many nonlinear neighbour graph-based dimension reduction algorithms, such as t-SNE [3], have been developed over the years to overcome this limitation"

In addition, the `cosine` has been used as a metric. The justification for this is that, given two vectors *a* and *b* of dimensionality *d*, the similarity between them can be calculated by this formula:

$$
similarity(a,b) = \cos(\theta) = \frac{\mathbf{a} \cdot \mathbf{b}}{\|\mathbf{a}\| \|\mathbf{b}\|}
$$


The main reason for using the cosine of the angle formed between the two vectors is that, the value of cos(θ) varies between:
- **1**: when the vectors are perfectly aligned (angle of 0∘).
- **0**: when the vectors are orthogonal (angle of 90∘).
- **-1**: when the vectors are completely opposite (180∘ angle).

From this formula, with UMAP we will be grouping according to the semantics between the different embeddings, providing us with a better solution.

As for the `n_neighbors` parameter, if we use small values, the representation of the data will focus on the local relationships, while a larger value will cause it to focus more on the global structure of the data.

## Clustering

In [ ]:
from sklearn.metrics import silhouette_score

min_cluster_sizes = range(10, 100, 10)
silhouette_scores = []

for size in min_cluster_sizes:
    cluster = HDBSCAN(min_cluster_size=size, metric='euclidean', prediction_data=True)
    cluster.fit(umap_model.fit_transform(embeddings))
    silhouette_scores.append(silhouette_score(embeddings, cluster.labels_))

plt.plot(min_cluster_sizes, silhouette_scores)
plt.title("Silhouette Score vs. Min Cluster Size")
plt.xlabel("Min Cluster Size")
plt.ylabel("Silhouette Score")
plt.show()

In [ ]:
# Get the min_cluster_size based on the silhouette_score
min_cluster_size = silhouette_scores.index(max(silhouette_scores))*10 + 10
print(f"Min cluster size: {min_cluster_size} (Empirically the best value is 15)")

# Modify min_cluster_size value based on experience
min_cluster_size = 15

cluster_model = HDBSCAN(min_cluster_size=min_cluster_size, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

**HDBSCAN (Hierarchical Density-Based Spatial Clustering of Applications with Noise)** is a clustering algorithm that stands out for its robustness and flexibility, especially on complex data such as tweets. Among some of its features of interest for this problem are the following:

- **No number of clusters required**: A strength of **HDBSCAN** is that it does not require the number of clusters to be specified in advance. The algorithm discovers the data structure autonomously. This is ideal for this problem, since, in principle, it is not known in advance how many topics (clusters) exist in the tweets.

- **Identification of outliers**: Points that do not clearly belong to any cluster are identified as noise. This allows to detect tweets that are not too relevant, which is useful for cleaning and analysis.

- **Variable density clusters**: **HDBSCAN** can identify clusters with different densities, this allows it to adapt to the reality of text data where topics may have different density levels.

As for **K-means**, it is a popular clustering algorithm known for its simplicity and computational efficiency, but it has certain limitations. Among the features of this algorithm of importance for this problem are the following:

- **Simplicity, computational efficiency and scalability**: In certain contexts, this feature may be of vital importance, however, for this problem (we have a not very high number of tweets), it is not a great advantage.

- **Assume uniform density**: **K-means** assumes that the clusters have similar density. This algorithm groups the data by trying to separate samples into *n* clusters of equal variance, minimizing a criterion known as the *incerity* or sum of squares within the cluster.
$$\sum_{i=0}^{n}\min_{\mu_j \in C}(||x_i - \mu_j||^2)$$

- **Noise sensitivity**: It is very sensitive to outliers and noise in the data. Outliers can significantly affect the position of cluster centroids, which can lead to suboptimal results.

The choice among the HDBSCAN parameters was made for the following reasons:

1. **Metric**: This parameter defines the distance metric used to calculate the distance between points. Some of the most commonly used values are:
    - `'euclidean'`: It has been decided to use this one because it is one of the most common and also, the dimensionality of the data is relatively low because *Dimensionality Reduction* has been previously done.

    - `'cosine'`: It is preferably used for high dimensionality data in which the angle of the vectors is more important than the absolute distance. For this reason, it has been decided to discard this metric.

2. **Cluster Selection Method**: This parameter determines how HDBSCAN selects the final clusters after the hierarchical clustering process. Among the options are:
    - `'eom'` (Excess of Mass): This method is typically used when you want to extract clusters that represent high density areas. The clusters are identified based on the criterion of “excess of mass”, which tries to identify which are the areas of higher density. It has been decided to use this method to try to better group tweets with the same theme.

    - `'leaf'`: This method chooses clusters based on the 'leaf' criterion, which means that it selects dense regions in the cluster tree where the hierarchy ends. This can often result in a more refined and less aggressive clustering process.

3. **Min Cluster Size**: The optimum value for this parameter was obtained empirically. A range between 10 and 100 with steps of 10 has been defined, the silhouette coefficient has been calculated and the value of *min_cluster_size* that offered the highest coefficient has been used. However, the most “optimal” *min_cluster_size* value according to silhouette produces very few *topics* and therefore very little useful information, with 15 being the best empirically tested value.

## Vectorizers

In [ ]:
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1,2))

There are mainly two types of vectorizers that can be used in BERTopic.

**CountVectorizer** is a common vectorizer that counts the frequency of words in a document. Some parameters have been adjusted such as:

1. **ngram_range**: Allows to include n-grams (sequences of n words) instead of just single words, which can help to better capture the context. Considering the domain of this problem, it has been decided to use a 2-word n-gram, enough to process structures of the type:
    - First Name + Last Name
    - First name + Verb
    - Organization + Verb
    - etc.

2. **stop_words**: Eliminates common words that do not provide relevant information such as determiners, prepositions, etc., improving the quality of the representation. In this case, as the tweets are in English, the value *"english ”* has been used.

**OnlineCountVectorizer** is an incremental variant of **CountVectorizer**, ideal for *streaming* data. For this problem it is not the most suitable.

In addition, the **CountVectorizer** can be applied before or after training the BERTopic model. We have decided to apply it before, since, it minimizes the size of the resulting **c-TF-IDF** matrix, which improves computational efficiency and reduces the amount of memory required.

## Topic representation

In [ ]:
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True, bm25_weighting=True)

**c-TF-IDF** is an adaptation of **TF-IDF** used to represent topics. Instead of calculating **TF-IDF** at the document level, **c-TF-IDF** does it at the *cluster* level (previously obtained), which makes it possible to identify what distinguishes the documents of a topic from those of others.

There are some parameters that we can adjust. These are as follows:

1. **bm25_weighting**: This boolean parameter allows to use a **BM-25** weighting method instead of the default method.

    The default weighting scheme is as follows:

    $$\log\left(1 + \frac{A}{f_x}\right)$$

    The BM-25 weighting method can be calculated as follows:

    $$\log\left(1 + \frac{A - {f_x} + 0.5}{f_x + 0.5}\right)$$

    It has been decided to set this parameter to `True` because, as the number of words in tweets is generally small, a better representation and topic generation is achieved.

2. **reduce_frequent_words**: This parameter reduces the influence of words that appear very frequently in all topics (but are not *stop words*).

    The default value of the frequency of a term is:
    
    $$|tf_{x,c}|$$

    If the value of this parameter is `True`, the frequency of a term is calculated by this formula:

    $$\sqrt{|tf_{x,c}|}$$

    It has been decided to set this parameter to `True` to try to give more importance to those words specific to the topic.

## Fine Tuning

In [ ]:
representation_model = MaximalMarginalRelevance(diversity = 0.3)

BERTopic offers several representation models to adjust the topic representations, which are not used by default, but are optional. These models can refine the *keywords* of the topics generated by c-TF-IDF. Some of the representation models offered are explained below.

**KeyBERTInspired** is a model that adjusts topic representations based on the semantic relationship between keywords and documents in each topic. It offers improved topic representations that take into account the structure of *clusters*.

**PartOfSpeech** is a model that takes into account the part of speech of keywords, extracting noun phrases from documents to generate new keyword candidates. It provides us with more relevant keywords by considering their grammatical function.

**MaximalMarginalRelevance (MMR)** is a model that reduces redundancy and improves keyword diversity by considering the similarity between keywords and the document, as well as the similarity between already selected keywords. It provides a more diverse set of keywords that maximizes the unique information in each topic.

Se ha decidido utilizar **MMR** para mejorar la diversidad de las keywords dentro de cada tema, reduciendo redundancias y haciendo más fácil la identificación de las características distintivas de cada tema. Se ha asignado `diversity = 0.3` de forma empírica.

## Model setup and training

In [ ]:
topic_model = BERTopic(
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=cluster_model,
  vectorizer_model=vectorizer_model,
  ctfidf_model=ctfidf_model,
  representation_model=representation_model,
  calculate_probabilities = True,
  min_topic_size = 50,
  n_gram_range=(1, 2),
  verbose = True,
  language='english'
)

In [ ]:
topics, probs = topic_model.fit_transform(train["tweet"])

## Visualization

In [ ]:
topic_model.visualize_topics()

In [ ]:
topics_per_class = topic_model.topics_per_class(train["tweet"], train["label"])
fig = topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=None)
fig.show()

In [ ]:
# As the dimensionality of the embeddings is 10, a redimensionality is made.
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
topic_model.visualize_documents(train["tweet"], reduced_embeddings=reduced_embeddings) # Not the best way as information is lost by reducing the dimensionality.

In [ ]:
topic_model.visualize_hierarchy()

In [ ]:
topic_model.visualize_barchart()

In [ ]:
topic_model.visualize_heatmap()

# Conclusion

**Put the conclusion here**

In addition to all the information and conclusions mentioned above, other possible questions have been raised whose study could be interesting and could help to differenciate more clearly between *real* and *fake* tweets. Some of these are the following:

- It could be interesting to extract tags from tweets (eg: #coronavirus) and study if there is any relationship between *fake* tweets and certain tags. This is because the use of hashtags is usually oriented to maximize the reach of tweets to a specific audience or to promote content related to specific topics, which in some cases could be associated with conspiracy groups or similar.

- Another possible idea related to the urls present in some tweets is to check the trustworthiness of the site using some heuristic between the reliability of the web page and the comments people have posted using some tools such as [MyWOT](https://www.mywot.com/scorecard).  Sometimes links to websites included in tweets can provide us with quite a bit of information. For example, some tweets contain a url that redirects to a web page called *[www.thespoof.com](www.thespoof.com)* on which there are untrustworthy fake news, political satire, and funny parody.

- It could be analyzed whether the citation of certain organizations or people influence whether a tweet is labeled as “real” or “fake”.